# Data Exploration

## Overview
This notebook performs data exploration and preprocessing for GOES images. The workflow includes data loading, quality control, temporal alignment.

### Libraries

In [ ]:
from glob import glob
from pathlib import Path
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

import pyproj
from scipy.ndimage import zoom

### Parameters

In [ ]:
LON, LAT = -74.0659, 4.6043   # Uniandes
IN_SIZE  = 224                # recorte n×n (par)
OUT_SIZE = 32                 # reescala final
CHANNELS = range(1, 17)       # C01..C16

FILES = sorted(glob("../data_raw/datos_GOES/OR_ABI-L2-MCMIPF-M6_G16_*.nc"))# [:200]  
OUT_DIR = Path("../data_interim/goes_dataset")
OUT_DIR.mkdir(parents=True, exist_ok=True)

### Functions

In [ ]:
def ensure_utc(ts_like):
    """Devuelve Timestamp tz-aware en UTC."""
    ts = pd.Timestamp(ts_like)
    return ts.tz_localize("UTC") if ts.tz is None else ts.tz_convert("UTC")

def resize_32(a):
    """Reescala a 32x32."""
    sy = OUT_SIZE / a.shape[0]
    sx = OUT_SIZE / a.shape[1]
    return zoom(a, (sy, sx), order=1).astype("float32")

def normalize(a, ch):
    """Normaliza: C01 visible → [0,1]; IR (180-330 K) → [0,1]."""
    a = a.astype("float32")
    if ch == 1:
        return np.clip(a, 0, 1)
    return (np.clip(a, 180, 330) - 180) / 150.0

def lonlat_to_geos_xy(ds, lon, lat):
    """(lon,lat) → (Xc,Yc)."""
    gip   = ds["goes_imager_projection"]
    h     = float(gip.perspective_point_height)
    lon_0 = float(gip.longitude_of_projection_origin)
    sweep = str(gip.sweep_angle_axis)
    geos  = pyproj.CRS.from_proj4(f"+proj=geos +h={h} +lon_0={lon_0} +sweep={sweep} +ellps=WGS84")
    wgs   = pyproj.CRS.from_epsg(4326)
    P     = pyproj.Transformer.from_crs(wgs, geos, always_xy=True)
    return P.transform(lon, lat)  # (Xc, Yc)

### Processing

In [ ]:
rows_patches = []   # cada fila: {'time': ts, '1': arr32, ..., '16': arr32}
rows_features = []  # cada fila: {'time': ts, 'C01_mean':..., 'C01_std':..., ...}

for nc_path in FILES:
    ds = xr.open_dataset(nc_path)

    # Ejes x,y (rad) → metros
    sat_h = float(ds["goes_imager_projection"].perspective_point_height)
    x = ds["x"].values * sat_h
    y = ds["y"].values * sat_h

    # Timestamp del archivo
    ts_attr = (
        ds.attrs.get("time_coverage_start")
        or ds.attrs.get("t")
        or next((ds[v].attrs.get("time_coverage_start")
                    for v in ds.data_vars if "time_coverage_start" in ds[v].attrs), None)
    )
    if ts_attr is None:
        ds.close()
        continue
    ts = ensure_utc(ts_attr)

    # Centro ROI → índices más cercanos
    Xc, Yc = lonlat_to_geos_xy(ds, LON, LAT)
    ix = int(np.argmin((x - Xc)**2))
    iy = int(np.argmin((y - Yc)**2))

    half = IN_SIZE // 2
    xs = slice(max(ix-half, 0), min(ix+half, x.size))
    ys = slice(max(iy-half, 0), min(iy+half, y.size))

    # Extrae canales: recorta, pad si toca borde, reescala y normaliza
    chan_row = {}
    feat_row = {}
    for ch in CHANNELS:
        var = f"CMI_C{ch:02d}"
        if var not in ds:
            arr32 = np.full((OUT_SIZE, OUT_SIZE), np.nan, np.float32)
            chan_row[str(ch)] = arr32
            feat_row[f"C{ch:02d}_mean"] = np.nan
            feat_row[f"C{ch:02d}_std"]  = np.nan
            continue

        patch = ds[var].isel(x=xs, y=ys).values  # (y,x)
        ph, pw = patch.shape
        if (ph, pw) != (IN_SIZE, IN_SIZE):
            pad_y = max(0, IN_SIZE - ph)
            pad_x = max(0, IN_SIZE - pw)
            patch = np.pad(patch, ((0, pad_y), (0, pad_x)), mode="edge")

        arr32 = resize_32(patch)
        arr32 = normalize(arr32, ch)

        chan_row[str(ch)] = arr32
        feat_row[f"C{ch:02d}_mean"] = float(np.nanmean(arr32))
        feat_row[f"C{ch:02d}_std"]  = float(np.nanstd(arr32))

    ds.close()

    chan_row["time"] = ts
    rows_patches.append(chan_row)

    feat_row["time"] = ts
    rows_features.append(feat_row)

# if not rows_patches:
#     raise RuntimeError("No se generaron parches.")

In [ ]:
sat_raw  = pd.DataFrame(rows_patches).set_index("time").sort_index()   # columnas '1'..'16' (arrays 32×32)
sat_feat = pd.DataFrame(rows_features).set_index("time").sort_index()  # columnas Cxx_mean/std

print("sat_raw :", sat_raw.shape)
print("sat_feat:", sat_feat.shape)

### Persistance

In [ ]:
sat_raw.to_pickle(OUT_DIR / "goes_patches.pkl")
sat_feat.to_parquet(OUT_DIR / "goes_features.parquet", engine="pyarrow", compression="zstd")
print("✅ Guardado:", OUT_DIR / "goes_patches.pkl")
print("✅ Guardado:", OUT_DIR / "goes_features.parquet")

### Graphics

In [ ]:
first_ts = sat_raw.index[0]
X0 = np.stack([sat_raw.loc[first_ts, str(ch)] for ch in CHANNELS], axis=0)  # (16,32,32)

fig_mosaic, axes = plt.subplots(4,4, figsize=(8,8))
for i, ax in enumerate(axes.ravel(), start=1):
    ax.imshow(X0[i-1], vmin=0, vmax=1, cmap="cividis")
    ax.set_title(f"C{i:02d}", fontsize=8)
    ax.axis("off")
fig_mosaic.suptitle(f"GOES-16 — {first_ts.isoformat()}", y=0.92)
plt.tight_layout(); plt.show()

fig_mosaic.savefig(OUT_DIR / "mosaic_c01_c16.png", dpi=180)

In [ ]:
nc_demo = FILES[0]
ds_demo = xr.open_dataset(nc_demo)

sat_h = float(ds_demo["goes_imager_projection"].perspective_point_height)
x = ds_demo["x"].values * sat_h
y = ds_demo["y"].values * sat_h

Xc, Yc = lonlat_to_geos_xy(ds_demo, LON, LAT)
ix = int(np.argmin((x - Xc)**2))
iy = int(np.argmin((y - Yc)**2))

half = IN_SIZE // 2
xs = slice(max(ix-half, 0), min(ix+half, x.size))
ys = slice(max(iy-half, 0), min(iy+half, y.size))

var = "CMI_C09" if "CMI_C09" in ds_demo else list(ds_demo.data_vars)[0]
full = ds_demo[var].values
crop = ds_demo[var].isel(x=xs, y=ys).values
ph, pw = crop.shape
if (ph, pw) != (IN_SIZE, IN_SIZE):
    crop = np.pad(crop, ((0, max(0,IN_SIZE-ph)), (0, max(0,IN_SIZE-pw))), mode="edge")
resized = resize_32(crop)

xmin, xmax = x[xs].min(), x[xs].max()
ymin, ymax = y[ys].min(), y[ys].max()

vmin = np.nanpercentile(full, 5)
vmax = np.nanpercentile(full, 95)

In [ ]:
gip = ds_demo["goes_imager_projection"]
proj = ccrs.Geostationary(
    central_longitude=float(gip.longitude_of_projection_origin),
    satellite_height=float(gip.perspective_point_height),
    sweep_axis=str(gip.sweep_angle_axis)
)

import matplotlib.patches as mpatches
fig_map = plt.figure(figsize=(14,4))

In [ ]:
# 1) Escena completa + rectángulo ROI
ax1 = fig_map.add_subplot(1,3,1, projection=proj)
ax1.imshow(full, extent=(x.min(), x.max(), y.min(), y.max()),
            transform=proj, origin="upper", vmin=vmin, vmax=vmax, cmap="cividis")
ax1.add_feature(cfeature.COASTLINE, linewidth=0.8)
ax1.add_feature(cfeature.BORDERS, linewidth=0.6)
ax1.add_patch(mpatches.Rectangle((xmin, ymin), xmax-xmin, ymax-ymin,
                                    fill=False, edgecolor="red", linewidth=2, transform=proj))
ax1.set_title("Escena completa + ROI"); ax1.axis("off")

# 2) Recorte 224×224
ax2 = fig_map.add_subplot(1,3,2, projection=proj)
ax2.imshow(crop, extent=(xmin, xmax, ymin, ymax),
            transform=proj, origin="upper", cmap="cividis")
ax2.set_title("Recorte (224×224)"); ax2.axis("off")

# 3) Reescala 32×32
ax3 = fig_map.add_subplot(1,3,3)
ax3.imshow(resized, cmap="cividis")
ax3.set_title("Reescala (32×32)"); ax3.axis("off")

plt.tight_layout(); plt.show()
fig_map.savefig(OUT_DIR / "full_roi_crop_rescale.png", dpi=180)

ds_demo.close()

### Diagnosis

In [ ]:
mean_cols = [c for c in sat_feat.columns if c.endswith("_mean")]
cover = pd.Series({c: sat_feat[c].notna().mean() for c in mean_cols}).sort_values(ascending=False)
print("Cobertura por canal (features *_mean):")
display(cover.to_frame("coverage").head(10))

best_mean = cover.index[0] if len(cover) else None
if best_mean and sat_feat[best_mean].dropna().shape[0] > 0:
    ax = sat_feat[best_mean].dropna().plot(figsize=(10,3), lw=1.0,
                                           title=f"{best_mean} (norm) — serie temporal")
    ax.grid(True, ls="--", alpha=0.4); plt.tight_layout(); plt.show()
    plt.gcf().savefig(OUT_DIR / "ts_best_mean.png", dpi=180)
else:
    print("No hay series válidas para graficar (todo NaN).")

### Export

In [ ]:
year = sat_raw.index[0].year

full_grid = pd.date_range(f"{year}-01-01 00:00:00", f"{year}-12-31 23:50:00", freq="10min", tz="UTC")

df_join = pd.DataFrame(index=full_grid).join(sat_raw, how="left")

# Alias 0..15 (si fuera necesario para compatibilidad)
for i in range(16):
    src, dst = str(i+1), str(i)
    if src in df_join.columns and dst not in df_join.columns:
        df_join[dst] = df_join[src]

out_pkl = OUT_DIR / "datos_sateli.pkl"
df_join.to_pickle(out_pkl)
print("✅ Guardado (formato legado):", out_pkl, "| shape:", df_join.shape)